In [1]:
pip install pandas plotly dash dash-bootstrap-components openpyxl

   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.9 MB 372.4 kB/s eta 0:00:20
   -- ------------------------------------- 0.5/7.9 MB 372.4 kB/s eta 0:00:20
   -- ------------------------------------- 0.5/7.9 MB 372.4 kB/s eta 0:00:20
   --- ------------------------------------ 0.8/7.9 MB 392.6 kB/s eta 0:00:19
   --- ------------------------------------ 0.8/7.9 MB 392.6 kB/s e

In [12]:
# 安装必要库（如果未安装）
# pip install pandas plotly dash dash-bootstrap-components

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc

# ==================== 基础历史数据 ====================
# 1. 各地区历史收入数据
regions_data = {
    '地区': ['美国', '中国', '欧洲', '亚太', '中东', '其他'],
    '2022': [405.53, 181.45, 80, 40, 15, 92.64],
    '2023': [452.8, 251.01, 100.41, 55.78, 20.08, 87.82],
    '2024': [438, 250.24, 104.26, 62.56, 20.85, 101.15],
}

regions_df = pd.DataFrame(regions_data)

# ==================== 不同情景的增长率假设 ====================
# 保守情景
growth_conservative = {
    '地区': ['美国', '中国', '欧洲', '亚太', '中东', '其他'],
    '2025增长率': [0.02, 0.05, 0.08, 0.15, 0.10, 0.04],
    '2026-2030 CAGR': [0.02, 0.05, 0.08, 0.15, 0.10, 0.04]
}

# 正常情景
growth_normal = {
    '地区': ['美国', '中国', '欧洲', '亚太', '中东', '其他'],
    '2025增长率': [0.04, 0.07, 0.12, 0.20, 0.15, 0.064],
    '2026-2030 CAGR': [0.04, 0.07, 0.12, 0.20, 0.15, 0.064]
}

# 乐观情景
growth_optimistic = {
    '地区': ['美国', '中国', '欧洲', '亚太', '中东', '其他'],
    '2025增长率': [0.06, 0.10, 0.18, 0.30, 0.20, 0.10],
    '2026-2030 CAGR': [0.06, 0.10, 0.18, 0.30, 0.20, 0.10]
}

growth_conservative_df = pd.DataFrame(growth_conservative)
growth_normal_df = pd.DataFrame(growth_normal)
growth_optimistic_df = pd.DataFrame(growth_optimistic)

# ==================== 生成各地区预测数据 ====================
def generate_region_forecast(base_df, growth_df, scenario_name):
    """生成地区收入预测"""
    forecast_years = ['2025', '2026', '2027', '2028', '2029', '2030']
    forecasts = []
    
    for _, region_row in growth_df.iterrows():
        region = region_row['地区']
        region_data = {'地区': region}
        
        # 获取2024年基础数据
        base_value = base_df[base_df['地区'] == region]['2024'].values[0]
        
        # 2025年预测
        growth_2025 = region_row['2025增长率']
        forecast_2025 = base_value * (1 + growth_2025)
        region_data['2025'] = round(forecast_2025, 2)
        
        # 2026-2030年预测
        cagr = region_row['2026-2030 CAGR']
        current_value = forecast_2025
        
        for i, year in enumerate(forecast_years[1:], 1):
            # 每年应用增长率
            current_value = current_value * (1 + cagr)
            region_data[year] = round(current_value, 2)
        
        # 添加情景标识
        region_data['情景'] = scenario_name
        forecasts.append(region_data)
    
    return pd.DataFrame(forecasts)

# 生成不同情景的预测
regions_forecast_conservative = generate_region_forecast(regions_df, growth_conservative_df, '保守')
regions_forecast_normal = generate_region_forecast(regions_df, growth_normal_df, '正常')
regions_forecast_optimistic = generate_region_forecast(regions_df, growth_optimistic_df, '乐观')

# 合并所有情景的预测
all_region_forecasts = pd.concat([
    regions_forecast_conservative,
    regions_forecast_normal,
    regions_forecast_optimistic
], ignore_index=True)

# ==================== 业务预测数据 ====================
years = ['2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']

# 历史业务数据
historical_business = {
    '汽车业务': [714.62, 824.19, 770.7],
    '能源业务': [39.09, 60.35, 100.86],
    '服务业务': [60.91, 83.19, 105.34],
    '传统业务总计': [814.62, 967.73, 976.9]
}

# 不同情景的业务增长率
def generate_business_forecast(scenario):
    """生成业务预测"""
    if scenario == '保守':
        rates = {
            '汽车业务': 0.03,
            '能源业务': 0.20,
            '服务业务': 0.15
        }
        new_business = {
            '2026': {'Optimus': 3, 'Robotaxi': 0},
            '2027': {'Optimus': 15, 'Robotaxi': 3},
            '2028': {'Optimus': 60, 'Robotaxi': 50},
            '2029': {'Optimus': 120, 'Robotaxi': 80},
            '2030': {'Optimus': 180, 'Robotaxi': 120}
        }
    elif scenario == '正常':
        rates = {
            '汽车业务': 0.04,
            '能源业务': 0.30,
            '服务业务': 0.20
        }
        new_business = {
            '2026': {'Optimus': 3, 'Robotaxi': 0},
            '2027': {'Optimus': 20, 'Robotaxi': 5},
            '2028': {'Optimus': 90, 'Robotaxi': 80},
            '2029': {'Optimus': 200, 'Robotaxi': 130},
            '2030': {'Optimus': 300, 'Robotaxi': 200}
        }
    else:  # 乐观
        rates = {
            '汽车业务': 0.06,
            '能源业务': 0.40,
            '服务业务': 0.25
        }
        new_business = {
            '2026': {'Optimus': 5, 'Robotaxi': 2},
            '2027': {'Optimus': 30, 'Robotaxi': 10},
            '2028': {'Optimus': 120, 'Robotaxi': 120},
            '2029': {'Optimus': 280, 'Robotaxi': 200},
            '2030': {'Optimus': 400, 'Robotaxi': 300}
        }
    
    # 传统业务预测
    traditional = {
        '年份': years,
        '汽车业务': [],
        '能源业务': [],
        '服务业务': [],
        '传统业务总计': []
    }
    
    # 添加历史数据
    for i in range(3):
        traditional['汽车业务'].append(historical_business['汽车业务'][i])
        traditional['能源业务'].append(historical_business['能源业务'][i])
        traditional['服务业务'].append(historical_business['服务业务'][i])
        traditional['传统业务总计'].append(historical_business['传统业务总计'][i])
    
    # 预测数据
    last_auto = historical_business['汽车业务'][-1]
    last_energy = historical_business['能源业务'][-1]
    last_service = historical_business['服务业务'][-1]
    
    for year in years[3:]:
        # 计算传统业务
        year_idx = years.index(year)
        auto_value = last_auto * ((1 + rates['汽车业务']) ** (year_idx - 2))
        energy_value = last_energy * ((1 + rates['能源业务']) ** (year_idx - 2))
        service_value = last_service * ((1 + rates['服务业务']) ** (year_idx - 2))
        
        traditional['汽车业务'].append(round(auto_value, 2))
        traditional['能源业务'].append(round(energy_value, 2))
        traditional['服务业务'].append(round(service_value, 2))
        traditional['传统业务总计'].append(round(auto_value + energy_value + service_value, 2))
    
    traditional_df = pd.DataFrame(traditional)
    traditional_df['情景'] = scenario
    
    # 新业务预测
    new_df = pd.DataFrame({
        '年份': years,
        'Optimus': [0, 0, 0, 0, 0, 0, 0, 0, 0],
        'Robotaxi': [0, 0, 0, 0, 0, 0, 0, 0, 0]
    })
    
    for year, values in new_business.items():
        if year in new_df['年份'].values:
            idx = new_df[new_df['年份'] == year].index[0]
            new_df.at[idx, 'Optimus'] = values['Optimus']
            new_df.at[idx, 'Robotaxi'] = values['Robotaxi']
    
    new_df['新业务总计'] = new_df['Optimus'] + new_df['Robotaxi']
    new_df['情景'] = scenario
    
    return traditional_df, new_df

# 生成业务预测
traditional_conservative, new_conservative = generate_business_forecast('保守')
traditional_normal, new_normal = generate_business_forecast('正常')
traditional_optimistic, new_optimistic = generate_business_forecast('乐观')

# 合并业务数据
all_traditional = pd.concat([traditional_conservative, traditional_normal, traditional_optimistic])
all_new_business = pd.concat([new_conservative, new_normal, new_optimistic])

# ==================== 创建Dash应用 ====================
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "特斯拉财务预测可视化仪表板 - 多情景分析"

# 布局设计
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("特斯拉财务预测可视化仪表板 - 多情景分析", 
                   className="text-center my-4 text-primary"),
            html.Hr()
        ])
    ]),
    
    # 控制面板
    dbc.Row([
        dbc.Col([
            html.Label("选择预测情景:", className="fw-bold"),
            dcc.RadioItems(
                id='scenario-selector',
                options=[
                    {'label': '保守情景', 'value': '保守'},
                    {'label': '正常情景', 'value': '正常'},
                    {'label': '乐观情景', 'value': '乐观'}
                ],
                value='正常',
                inline=True,
                className="mb-4",
                labelStyle={'margin-right': '20px'}
            )
        ], width=6),
        
        dbc.Col([
            html.Label("选择分析年份:", className="fw-bold"),
            dcc.Dropdown(
                id='year-selector',
                options=[{'label': year, 'value': year} for year in years],
                value='2030',
                clearable=False,
                className="mb-4"
            )
        ], width=3),
        
        dbc.Col([
            html.Label("选择地区:", className="fw-bold"),
            dcc.Dropdown(
                id='region-selector',
                options=[{'label': region, 'value': region} for region in regions_df['地区']],
                value='美国',
                clearable=False,
                className="mb-4"
            )
        ], width=3)
    ], className="mb-4"),
    
    # 图表区域
    dbc.Tabs([
        dbc.Tab([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id='total-revenue-chart'),
                ], width=12, className="mb-4"),
            ]),
            
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id='business-composition-chart'),
                ], width=6),
                
                dbc.Col([
                    dcc.Graph(id='region-comparison-chart'),
                ], width=6),
            ], className="mb-4"),
        ], label="总体概览"),
        
        dbc.Tab([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id='region-growth-chart'),
                ], width=12, className="mb-4"),
            ]),
            
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id='cagr-comparison-chart'),
                ], width=12, className="mb-4"),
            ]),
        ], label="地区分析"),
        
        dbc.Tab([
            dbc.Row([
                dbc.Col([
                    html.H4("各地区收入数据", className="mt-4"),
                    html.Div(id='region-data-table')
                ], width=12)
            ])
        ], label="数据表格"),
    ]),
], fluid=True)

# ==================== 回调函数 ====================
@app.callback(
    Output('total-revenue-chart', 'figure'),
    Input('scenario-selector', 'value')
)
def update_total_revenue(scenario):
    """更新总收入趋势图"""
    fig_total = go.Figure()
    
    # 获取当前情景的数据
    traditional_df = all_traditional[all_traditional['情景'] == scenario]
    new_df = all_new_business[all_new_business['情景'] == scenario]
    
    # 传统业务
    fig_total.add_trace(go.Scatter(
        x=years,
        y=traditional_df['传统业务总计'],
        mode='lines+markers',
        name='传统业务',
        line=dict(color='blue', width=3),
        marker=dict(size=8)
    ))
    
    # 新增业务
    fig_total.add_trace(go.Scatter(
        x=years,
        y=new_df['新业务总计'],
        mode='lines+markers',
        name='新增业务',
        line=dict(color='green', width=3),
        marker=dict(size=8)
    ))
    
    # 总收入
    total_revenue = [t + n for t, n in zip(traditional_df['传统业务总计'], new_df['新业务总计'])]
    fig_total.add_trace(go.Scatter(
        x=years,
        y=total_revenue,
        mode='lines+markers',
        name='总收入',
        line=dict(color='red', width=4, dash='dash'),
        marker=dict(size=10)
    ))
    
    # 添加增长率标注
    for i in range(1, len(years)):
        growth_rate = ((total_revenue[i] - total_revenue[i-1]) / total_revenue[i-1] * 100)
        if growth_rate > 0:
            fig_total.add_annotation(
                x=years[i],
                y=total_revenue[i],
                text=f"+{growth_rate:.1f}%",
                showarrow=False,
                yshift=20,
                font=dict(size=10, color='black')
            )
    
    fig_total.update_layout(
        title=f'{scenario}情景总收入预测趋势（2022-2030，亿美元）',
        xaxis_title='年份',
        yaxis_title='收入（亿美元）',
        hovermode='x unified',
        template='plotly_white',
        height=400
    )
    
    return fig_total

@app.callback(
    Output('business-composition-chart', 'figure'),
    [Input('year-selector', 'value'),
     Input('scenario-selector', 'value')]
)
def update_business_composition(selected_year, scenario):
    """更新业务构成图"""
    # 获取数据
    traditional_df = all_traditional[all_traditional['情景'] == scenario]
    new_df = all_new_business[all_new_business['情景'] == scenario]
    
    # 获取指定年份的数据
    if selected_year in traditional_df['年份'].values:
        trad_row = traditional_df[traditional_df['年份'] == selected_year].iloc[0]
        new_row = new_df[new_df['年份'] == selected_year].iloc[0]
        
        auto_value = trad_row['汽车业务']
        energy_value = trad_row['能源业务']
        service_value = trad_row['服务业务']
        optimus_value = new_row['Optimus']
        robotaxi_value = new_row['Robotaxi']
    else:
        auto_value = energy_value = service_value = optimus_value = robotaxi_value = 0
    
    # 准备饼图数据
    labels = ['汽车业务', '能源业务', '服务业务', 'Optimus', 'Robotaxi']
    values = [auto_value, energy_value, service_value, optimus_value, robotaxi_value]
    
    # 过滤掉值为0的业务
    filtered_data = [(l, v) for l, v in zip(labels, values) if v > 0]
    if filtered_data:
        filtered_labels, filtered_values = zip(*filtered_data)
    else:
        filtered_labels = ['无数据']
        filtered_values = [1]
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    fig_pie = go.Figure(data=[go.Pie(
        labels=filtered_labels,
        values=filtered_values,
        hole=0.4,
        marker=dict(colors=colors[:len(filtered_labels)]),
        textinfo='label+percent+value',
        texttemplate='%{label}<br>%{value:.1f}亿<br>(%{percent})',
        hovertemplate='<b>%{label}</b><br>收入: %{value:.1f}亿美元<br>占比: %{percent}<extra></extra>'
    )])
    
    fig_pie.update_layout(
        title=f'{selected_year}年{scenario}情景业务收入构成',
        template='plotly_white',
        height=400,
        showlegend=False
    )
    
    return fig_pie

@app.callback(
    Output('region-comparison-chart', 'figure'),
    [Input('year-selector', 'value'),
     Input('scenario-selector', 'value')]
)
def update_region_comparison(selected_year, scenario):
    """更新地区对比图"""
    fig = go.Figure()
    
    if selected_year in ['2022', '2023', '2024']:
        # 历史年份
        sorted_regions = regions_df.sort_values(by=selected_year, ascending=True)
        values = sorted_regions[selected_year].values
        labels = sorted_regions['地区'].values
    else:
        # 预测年份
        region_data = all_region_forecasts[
            (all_region_forecasts['情景'] == scenario)
        ]
        if not region_data.empty:
            sorted_regions = region_data.sort_values(by=selected_year, ascending=True)
            values = sorted_regions[selected_year].values
            labels = sorted_regions['地区'].values
        else:
            values = [0] * len(regions_df)
            labels = regions_df['地区'].values
    
    fig.add_trace(go.Bar(
        y=labels,
        x=values,
        orientation='h',
        marker=dict(
            color=values,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="收入（亿美元）")
        ),
        text=[f"{val:.1f}亿" for val in values],
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>收入: %{x:.1f}亿美元<extra></extra>'
    ))
    
    fig.update_layout(
        title=f'{selected_year}年{scenario}情景各地区收入对比',
        xaxis_title='收入（亿美元）',
        yaxis_title='地区',
        template='plotly_white',
        height=400
    )
    
    return fig

@app.callback(
    Output('region-growth-chart', 'figure'),
    [Input('region-selector', 'value'),
     Input('scenario-selector', 'value')]
)
def update_region_growth(selected_region, scenario):
    """更新地区增长趋势图"""
    fig = go.Figure()
    
    # 获取历史数据
    hist_years = ['2022', '2023', '2024']
    hist_values = [
        regions_df[regions_df['地区'] == selected_region][year].values[0]
        for year in hist_years
    ]
    
    # 获取预测数据
    forecast_years = ['2025', '2026', '2027', '2028', '2029', '2030']
    region_data = all_region_forecasts[
        (all_region_forecasts['地区'] == selected_region) & 
        (all_region_forecasts['情景'] == scenario)
    ]
    
    if not region_data.empty:
        forecast_values = [
            region_data[year].values[0] for year in forecast_years
        ]
    else:
        forecast_values = [0] * len(forecast_years)
    
    # 合并数据
    all_years = hist_years + forecast_years
    all_values = hist_values + forecast_values
    
    fig.add_trace(go.Scatter(
        x=all_years,
        y=all_values,
        mode='lines+markers',
        name=selected_region,
        line=dict(color='orange', width=4),
        marker=dict(size=10),
        text=[f"{val:.1f}" for val in all_values],
        textposition='top center'
    ))
    
    # 添加增长率标注
    for i in range(1, len(all_years)):
        growth_rate = ((all_values[i] - all_values[i-1]) / all_values[i-1] * 100)
        if abs(growth_rate) > 0.1:  # 只显示显著变化
            fig.add_annotation(
                x=all_years[i],
                y=all_values[i],
                text=f"↑{growth_rate:.1f}%" if growth_rate > 0 else f"↓{abs(growth_rate):.1f}%",
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor='green' if growth_rate > 0 else 'red',
                ax=0,
                ay=-30,
                font=dict(size=10)
            )
    
    fig.update_layout(
        title=f'{selected_region}地区{scenario}情景收入预测（亿美元）',
        xaxis_title='年份',
        yaxis_title='收入（亿美元）',
        template='plotly_white',
        height=400
    )
    
    return fig

@app.callback(
    Output('cagr-comparison-chart', 'figure'),
    Input('scenario-selector', 'value')
)
def update_cagr_comparison(scenario):
    """更新CAGR对比图"""
    fig = go.Figure()
    
    # 选择对应的增长率数据
    if scenario == '保守':
        growth_df = growth_conservative_df
        title = '保守情景各地区2026-2030年复合增长率'
        color = 'gray'
    elif scenario == '正常':
        growth_df = growth_normal_df
        title = '正常情景各地区2026-2030年复合增长率'
        color = 'blue'
    else:
        growth_df = growth_optimistic_df
        title = '乐观情景各地区2026-2030年复合增长率'
        color = 'green'
    
    fig.add_trace(go.Bar(
        x=growth_df['地区'],
        y=growth_df['2026-2030 CAGR'] * 100,
        marker_color=color,
        text=[f"{val*100:.1f}%" for val in growth_df['2026-2030 CAGR']],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>CAGR: %{y:.1f}%<extra></extra>'
    ))
    
    fig.update_layout(
        title=title,
        yaxis_title='CAGR (%)',
        xaxis_title='地区',
        template='plotly_white',
        height=400,
        yaxis=dict(tickformat='.1f%')
    )
    
    return fig

@app.callback(
    Output('region-data-table', 'children'),
    [Input('year-selector', 'value'),
     Input('scenario-selector', 'value')]
)
def update_data_table(selected_year, scenario):
    """更新数据表格"""
    # 准备表格数据
    if selected_year in ['2022', '2023', '2024']:
        # 历史年份
        table_data = regions_df[['地区', '2022', '2023', '2024']].copy()
        table_data[selected_year] = regions_df[selected_year]
    else:
        # 预测年份
        region_data = all_region_forecasts[
            (all_region_forecasts['情景'] == scenario)
        ]
        
        table_data = regions_df[['地区', '2022', '2023', '2024']].copy()
        
        if not region_data.empty and selected_year in region_data.columns:
            for _, region_row in region_data.iterrows():
                region = region_row['地区']
                value = region_row[selected_year]
                table_data.loc[table_data['地区'] == region, selected_year] = value
    
    # 计算增长率
    if '2022' in table_data.columns and selected_year in table_data.columns:
        table_data['增长率(22-选中年)'] = ((table_data[selected_year] - table_data['2022']) / table_data['2022'] * 100).round(1)
    
    # 重新排序列
    cols = ['地区', '2022', '2023', '2024']
    if selected_year not in cols:
        cols.append(selected_year)
    if '增长率(22-选中年)' in table_data.columns:
        cols.append('增长率(22-选中年)')
    
    table_data = table_data[cols]
    
    # 创建表格
    table = dbc.Table(
        [
            html.Thead(html.Tr([html.Th(col) for col in table_data.columns])),
            html.Tbody([
                html.Tr([
                    html.Td(f"{table_data.iloc[i][col]:.1f}" if isinstance(table_data.iloc[i][col], (int, float)) else str(table_data.iloc[i][col])) 
                    for col in table_data.columns
                ]) for i in range(len(table_data))
            ])
        ],
        bordered=True,
        hover=True,
        responsive=True,
        striped=True,
        className="mt-2"
    )
    
    return table

# ==================== 运行应用 ====================
if __name__ == '__main__':
    print("启动特斯拉财务预测多情景分析仪表板...")
    print("访问 http://127.0.0.1:8050 查看仪表板")
    app.run(debug=True, port=8050)

启动特斯拉财务预测多情景分析仪表板...
访问 http://127.0.0.1:8050 查看仪表板
